In [1]:
### MBTI Personality Classifier


In [2]:
## Importing libraries
import pandas as pd
import numpy as np
import math

In [3]:
## FUNCTION : 
# We cannot process string values in ML, that's why we need to encode the last column with integers. Additionally, we will need string values when we finish tha classification...
"""
encoding_list = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
decoding_list = ['ESTJ','ENTJ','ESFJ','ENFJ','ISTJ','ISFJ','INTJ','INFJ','ESTP','ESFP','ENTP','ENFP','ISTP','ISFP','INTP','INFP']

for i in encoding_list:
    base_data.loc[base_data["Personality"] == decoding_list[i], "Personality"] = i
base_data.head()
"""
### crawl from TXT
def encode_pers_types(df):
    with open('16p-Mapping.txt') as f:
        lines = f.readlines()
    mappin = lines[-16:]

    for i in range(len(mappin)):
        pers_type = mappin[i][-5:-1]
        pers_type_id = mappin[i][0:-6]
        df.loc[df["Personality"] == pers_type, "Personality"] = pers_type_id
        
    return base_data

In [4]:
with open('16p-Mapping.txt') as f:
    lines = f.readlines()
mappin = lines[-16:]

for i in range(len(mappin)):
    pers_type = mappin[i][-5:-1]
    pers_type_id = mappin[i][0:-6]
    print(pers_type_id)
pers_type_id

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


'15'

In [5]:
## FUNCTION :
## Feature Normalization
# In order to have better results, we normalize the data
"""
# We can create a function to normalize all data in columns, then we can call this function for all columns.
def normalize_column(df,column):
    max_x = np.array(df[column].values.tolist()).max()
    min_x = np.array(df[column].values.tolist()).min()
    rang = max_x-min_x
    for i in range(len(df[column])):
        df.loc[i,column] = (df[column][i]-min_x)/rang    
    return df
# 
normalized_data = train_data
for col in train_data.columns:
    normalized_data = normalize_column(normalized_data,col)
"""
# While the previous method was okey for different dataframes but we have a dataframe almaost all columns has the same range and values (-3,-2,-1,0,1,2,3). 
# Therefore we can update the values manually for less computational effort
def normalize_data(df):
    value_list = [-3,-2,-1,0,1,2,3]
    max_x = max(value_list)
    min_x = min(value_list)
    rang = max_x-min_x
    normalized_value_list = list(map(lambda x: (x-min_x)/rang, value_list))
    # in each column, for each cell we update to normalized value 
    for col in df.columns:
        for i in range(len(value_list)):
            df.loc[df[col] == value_list[i], col] = normalized_value_list[i]          
    return df

In [6]:
## FUNCTION : 

def fold_dataframe(base_data,test_set_letter):
    k_fold_number = 5
    period = math.floor(len(base_data)/k_fold_number)
    fold_A = base_data.iloc[:1*period+1,:]
    fold_B = base_data.iloc[1*period+1:2*period+1,:]
    fold_C = base_data.iloc[2*period+1:3*period+1,:]
    fold_D = base_data.iloc[3*period+1:4*period+1,:]
    fold_E = base_data.iloc[4*period+1:,:]
    if test_set_letter == 'A':
        folded_df = pd.concat([fold_E, fold_B, fold_C, fold_D, fold_A], axis=0)
    elif test_set_letter == 'B':
        folded_df = pd.concat([fold_A, fold_E, fold_C, fold_D, fold_B], axis=0)
    elif test_set_letter == 'C':
        folded_df = pd.concat([fold_A, fold_B, fold_E, fold_D, fold_C], axis=0)
    elif test_set_letter == 'D':
        folded_df = pd.concat([fold_A, fold_B, fold_C, fold_E, fold_D], axis=0)
    elif test_set_letter == 'E':
        folded_df = pd.concat([fold_A, fold_B, fold_C, fold_D, fold_E], axis=0)
    return folded_df

In [7]:
## FUNCTION : 

## Dataset partitioning
def partition_df(base_data):
    normalized_data = base_data
    test_train_rate = 80/100
    limit = math.floor(len(normalized_data)*test_train_rate)
    train_x_data = normalized_data.iloc[0:limit+1,0:60]
    train_y_data = normalized_data.iloc[0:limit+1,60:61]
    test_x_data = normalized_data.iloc[limit+1:len(normalized_data),0:60]
    test_y_data = normalized_data.iloc[limit+1:len(normalized_data),60:61]

    ## dataframe to numpy array
    train_x = train_x_data.to_numpy()
    train_y = train_y_data.to_numpy()
    test_x = test_x_data.to_numpy()
    test_y = test_y_data.to_numpy()
    return [train_x,train_y,test_x,test_y]

In [8]:
## FUNCTION : 

### Distance Metric -trial1
"""
def calculate_distance(point1,point2):
    dist = 0
    for i in range(len(point1)):
        dist = dist + (point1[i]-point2[i])**2
    dist = dist**(1/2)
    return dist
# calculate_distance([0,0],[3,4]) it should return 5
"""
### Distance Metric -trial2
"""
def calculate_distances(point1,train_points):
    # this function takes a points coordinates as a list and train points' coordinates as a matrix
    # and return distance list
    m= len(train_points)
    point1_matrix = np.tile(point1,(m,1))
    print(point1_matrix.shape)
    # subtracting matrix
    fark = train_points - point1_matrix
    # dot product a1.b1 a2.b2 but this will create a1.b2 and a2.b1 therefore we will select only diagonal values
    sqsum = np.dot(fark, fark.T)
    # squareroot 
    dist = np.sqrt(sqsum).diagonal()
    return dist
# calculate_distances([0,0],[[3,4],[5,12]]) it should return a list as [5,13]
"""
### Distance Metric -trial3
def calculate_distances(point1,train_points):
    # this function takes a points coordinates as a list and train points' coordinates as a matrix
    # and return distance list -previous method was again cause complexity issues
    distances = np.linalg.norm(train_points - point1, axis=1) # distances between points can be computed with the norm of the difference vector
    return distances
# calculate_distances(np.array([0,0]),np.array([[3,4],[5,12]])) it should return a list as [5.,13.]

In [9]:
## FUNCTION : 

### kNN classifier
def knn_classify(train_x,train_y,test_x,k):

    # We should decide how we assign class labels: Majority vote or plurality voting. 
    # There are 16 classes, therefore we will use plurality voting and required vote rate will be 100/16 % + = %6.25+
    # What if there 2 options have rate greater than 6.25.. 

    # We will use Euclidean distance with p=2 as distance metric.
    predicted_list = [] #np.zeros(len(test_x))
    for test_row in test_x: # len(test_x) 
        #test_row = test_x[test_row_index:test_row_index+1][0]
        dist_array = calculate_distances(test_row,train_x)
        # we will find the nearest k neighbours and their target values then we will determine the class for the test row
        # here we can use "argsort" also, however we will try to find them with loop
        """
        for nn in range(k):
            # getting the minimum value of distance list
            min_val = min(dist_array)
            # print(min_val)
            #finding the index of minimum value 
            min_ind = dist_array.index(min_val)
            # we will find the target value of this neighbour
            tvalue = train_y[min_ind][0]
            target_list.append(tvalue)
            # we should update the dist_array value to a huge value so as to not select again
            dist_array[min_ind] = max(dist_array)*99
        """
        nearest_neighbors_indeces = dist_array.argsort()[:k]
        nearest_neighbor_targets = train_y[nearest_neighbors_indeces]
        target_list = [item for sublist in nearest_neighbor_targets for item in sublist]

        # now, we need to find the most frequent element in the target value list
        most_freq_val = max(set(target_list), key = target_list.count) # here, if there are different target values which have same frequency we will choose the first element - it may result in misperception
        # print(target_list)
        # predicted_list[test_row_index] = most_freq_val
        predicted_list.append(most_freq_val)
    # When we finish predictioning for each test row, we return the prediction results
    return predicted_list

In [10]:
## FUNCTION : 

### Performance Evaluation Metrics Implementation
def error_analysis(test_y,predicted_list):
    misclassified_indeces = []
    confusion_matrix = np.zeros([16,16], dtype = int)
    for i in range(len(test_x)):
        true_value = int(test_y[i][0])
        prediction = int(predicted_list[i])
        # finding misclassified samples 
        if true_value - prediction != 0:
            misclassified_indeces.append(i)
        # creating confusion matrix
        confusion_matrix[prediction][true_value] = confusion_matrix[prediction][true_value] + 1
    # There are 16 different classes, therefore we will calculate evaluation metrics for each class
    evaluation_data = []
    for ptype in range(16):
        tp = confusion_matrix[ptype][ptype]
        tn = sum(confusion_matrix.diagonal())-tp        
        fp = sum(confusion_matrix[ptype])-tp
        fn = sum(confusion_matrix[:][ptype])-tp

        accuracy = (tp+tn)/(tp+tn+fp+fn)
        precision = (tp)/(tp+fp)
        recall = (tp)/(tp+fn)
        f1_score = 2*(precision*recall)/(precision+recall)
        ptype_dict = {'ptype1':ptype,'accuracy':accuracy,'precision':precision,'recall':recall,'f1_score':f1_score}
        evaluation_data.append(ptype_dict)
    # To have a generic evaluation metric, we add total_score
    total_tp = sum(confusion_matrix.diagonal())
    total_fp = sum(sum(confusion_matrix))-total_tp
    total_score = total_tp/(total_tp+total_fp)
    # returnin detailed evaluation data, total score, number of misclassified samples and their indices
    return {'total_score':total_score},evaluation_data,{'misclassification_amount':len(misclassified_indeces)},misclassified_indeces

In [11]:
### MAIN PART 

### kNN Algorithm Framewrok is READY TO RUN
## 0 Loading Data
rowdata = pd.read_csv('16P.csv',encoding='cp1252')
## 1 Shuffling Data
shuffled_data= rowdata.sample(frac=1) # function used: .sample()
## 2 Manipulating data
# We need to drop “Response Id” column, because it has no effect on classification
base_data = shuffled_data.drop(columns={'Response Id'})
# We cannot process string values in ML, that's why we need to encode the last column with integers. Additionally, we will need string values when we finish tha classification...
base_data = encode_pers_types(base_data) # function used: encode_pers_types()
## 3 Feature normalization (We will move on either with normalization and without normalization)
unnormalized_data = base_data
normalized_data = base_data
normalized_data.iloc[:,0:60] = normalize_data(normalized_data.iloc[:,0:60]) # function used: normalize_data()

In [12]:
### k = 1 | Normalized | K-fold: E
k = 1
K_letter = 'E'
input_data = normalized_data
## 4 Fold study and selecting fold to test
normalized_testE = fold_dataframe(normalized_data,K_letter) # function used: fold_dataframe()
## 5 Partitioning
[train_x,train_y,test_x,test_y] = partition_df(normalized_testE) # function used: partition_df()
## 6 Classification 
predictions_for_normalized_testE = knn_classify(train_x,train_y,test_x,k)


# 7 Error Analysis
print("RESULTS FOR k= ", k, ", with normalization and K-Fold:", K_letter)
detailed_error_analysis = error_analysis(test_y,predictions_for_normalized_testE)
print(detailed_error_analysis[0])


RESULTS FOR k=  1 , with normalization and K-Fold: E
{'total_score': 0.962080173347779}


In [13]:
### k = 3 | Normalized | K-fold: E
k = 3
K_letter = 'E'
input_data = normalized_data
## 4 Fold study and selecting fold to test
normalized_testE = fold_dataframe(normalized_data,K_letter) # function used: fold_dataframe()
## 5 Partitioning
[train_x,train_y,test_x,test_y] = partition_df(normalized_testE) # function used: partition_df()
## 6 Classification 
predictions_for_normalized_testE_k3 = knn_classify(train_x,train_y,test_x,k)

# 7 Error Analysis
print("RESULTS FOR k= ", k, ", with normalization and K-Fold:", K_letter)
detailed_error_analysis = error_analysis(test_y,predictions_for_normalized_testE_k3)
print(detailed_error_analysis[0])

RESULTS FOR k=  3 , with normalization and K-Fold: E
{'total_score': 0.9788315692974414}


In [14]:
### k = 5 | Normalized | K-fold: E
k = 5
K_letter = 'E'
input_data = normalized_data
## 4 Fold study and selecting fold to test
normalized_testE = fold_dataframe(normalized_data,K_letter) # function used: fold_dataframe()
## 5 Partitioning
[train_x,train_y,test_x,test_y] = partition_df(normalized_testE) # function used: partition_df()
## 6 Classification 
predictions_for_normalized_testE_k5 = knn_classify(train_x,train_y,test_x,k)

# 7 Error Analysis
print("RESULTS FOR k= ", k, ", with normalization and K-Fold:", K_letter)
detailed_error_analysis = error_analysis(test_y,predictions_for_normalized_testE_k5)
print(detailed_error_analysis[0])

RESULTS FOR k=  5 , with normalization and K-Fold: E
{'total_score': 0.9829152429369115}


In [15]:
### k = 7 | Normalized | K-fold: E
k = 7
K_letter = 'E'
input_data = normalized_data
## 4 Fold study and selecting fold to test
normalized_testE = fold_dataframe(normalized_data,K_letter) # function used: fold_dataframe()
## 5 Partitioning
[train_x,train_y,test_x,test_y] = partition_df(normalized_testE) # function used: partition_df()
## 6 Classification 
predictions_for_normalized_testE_k7 = knn_classify(train_x,train_y,test_x,k)

# 7 Error Analysis
print("RESULTS FOR k= ", k, ", with normalization and K-Fold:", K_letter)
detailed_error_analysis = error_analysis(test_y,predictions_for_normalized_testE_k7)
print(detailed_error_analysis[0])

RESULTS FOR k=  7 , with normalization and K-Fold: E
{'total_score': 0.9838319859988333}


In [16]:
#### number of misclassified samples and their indices...WHY?
print(detailed_error_analysis[2])
detailed_error_analysis[3]

{'misclassification_amount': 194}


[55,
 113,
 202,
 408,
 488,
 502,
 510,
 530,
 539,
 543,
 697,
 727,
 798,
 827,
 834,
 890,
 897,
 930,
 944,
 954,
 1048,
 1067,
 1070,
 1112,
 1149,
 1262,
 1299,
 1304,
 1329,
 1418,
 1437,
 1569,
 1614,
 1643,
 1671,
 1704,
 1730,
 1740,
 1892,
 1932,
 2034,
 2047,
 2071,
 2193,
 2241,
 2319,
 2324,
 2325,
 2334,
 2427,
 2468,
 2478,
 2479,
 2482,
 2536,
 2565,
 2644,
 2669,
 2755,
 2778,
 2824,
 2935,
 2940,
 2942,
 3133,
 3224,
 3411,
 3442,
 3515,
 3543,
 3706,
 3764,
 3768,
 3805,
 3839,
 3855,
 4011,
 4064,
 4269,
 4276,
 4367,
 4443,
 4502,
 4622,
 4640,
 4650,
 4663,
 4771,
 4840,
 4847,
 4920,
 4963,
 5141,
 5150,
 5308,
 5499,
 5510,
 5928,
 5993,
 5998,
 6033,
 6150,
 6218,
 6221,
 6263,
 6288,
 6324,
 6343,
 6385,
 6431,
 6436,
 6469,
 6577,
 6620,
 6647,
 6651,
 6994,
 7030,
 7036,
 7094,
 7101,
 7108,
 7182,
 7408,
 7551,
 7610,
 7641,
 7679,
 7688,
 7694,
 7807,
 7870,
 7871,
 7872,
 7906,
 7963,
 8082,
 8106,
 8125,
 8158,
 8331,
 8339,
 8357,
 8396,
 8398,
 8416,

In [17]:
### k = 9 | Normalized | K-fold: E
k = 9
K_letter = 'E'
input_data = normalized_data
## 4 Fold study and selecting fold to test
normalized_testE = fold_dataframe(normalized_data,K_letter) # function used: fold_dataframe()
## 5 Partitioning
[train_x,train_y,test_x,test_y] = partition_df(normalized_testE) # function used: partition_df()
## 6 Classification 
predictions_for_normalized_testE_k9 = knn_classify(train_x,train_y,test_x,k)


# 7 Error Analysis
print("RESULTS FOR k= ", k, ", with normalization and K-Fold:", K_letter)
detailed_error_analysis = error_analysis(test_y,predictions_for_normalized_testE_k9)
print(detailed_error_analysis[0])

RESULTS FOR k=  9 , with normalization and K-Fold: E
{'total_score': 0.9843320276689724}


In [18]:
### k = 7 | Normalized | K-fold: D
k = 7
K_letter = 'E'
input_data = normalized_data
## 4 Fold study and selecting fold to test
normalized_testE = fold_dataframe(normalized_data,K_letter) # function used: fold_dataframe()
## 5 Partitioning
[train_x,train_y,test_x,test_y] = partition_df(normalized_testE) # function used: partition_df()
## 6 Classification 
predictions_for_normalized_testD_k7 = knn_classify(train_x,train_y,test_x,k)

# 7 Error Analysis
print("RESULTS FOR k= ", k, ", with normalization and K-Fold:", K_letter)
detailed_error_analysis = error_analysis(test_y,predictions_for_normalized_testD_k7)
print(detailed_error_analysis[0])

RESULTS FOR k=  7 , with normalization and K-Fold: E
{'total_score': 0.9838319859988333}


In [19]:
### k = 7 | Normalized | K-fold: C
k = 7
K_letter = 'C'
input_data = normalized_data
## 4 Fold study and selecting fold to test
normalized_testE = fold_dataframe(normalized_data,K_letter) # function used: fold_dataframe()
## 5 Partitioning
[train_x,train_y,test_x,test_y] = partition_df(normalized_testE) # function used: partition_df()
## 6 Classification 
predictions_for_normalized_testC_k7 = knn_classify(train_x,train_y,test_x,k)

# 7 Error Analysis
print("RESULTS FOR k= ", k, ", with normalization and K-Fold:", K_letter)
detailed_error_analysis = error_analysis(test_y,predictions_for_normalized_testC_k7)
print(detailed_error_analysis[0])

RESULTS FOR k=  7 , with normalization and K-Fold: C
{'total_score': 0.9830819234936244}


In [20]:
### k = 7 | Normalized | K-fold: B
k = 7
K_letter = 'B'
input_data = normalized_data
## 4 Fold study and selecting fold to test
normalized_testE = fold_dataframe(normalized_data,K_letter) # function used: fold_dataframe()
## 5 Partitioning
[train_x,train_y,test_x,test_y] = partition_df(normalized_testE) # function used: partition_df()
## 6 Classification 
predictions_for_normalized_testB_k7 = knn_classify(train_x,train_y,test_x,k)


# 7 Error Analysis
print("RESULTS FOR k= ", k, ", with normalization and K-Fold:", K_letter)
detailed_error_analysis = error_analysis(test_y,predictions_for_normalized_testB_k7)
print(detailed_error_analysis[0])

RESULTS FOR k=  7 , with normalization and K-Fold: B
{'total_score': 0.9829152429369115}


In [21]:
### k = 7 | Normalized | K-fold: A
k = 7
K_letter = 'A'
input_data = normalized_data
## 4 Fold study and selecting fold to test
normalized_testE = fold_dataframe(normalized_data,K_letter) # function used: fold_dataframe()
## 5 Partitioning
[train_x,train_y,test_x,test_y] = partition_df(normalized_testE) # function used: partition_df()
## 6 Classification 
predictions_for_normalized_testA_k7 = knn_classify(train_x,train_y,test_x,k)

# 7 Error Analysis
print("RESULTS FOR k= ", k, ", with normalization and K-Fold:", K_letter)
detailed_error_analysis = error_analysis(test_y,predictions_for_normalized_testA_k7)
print(detailed_error_analysis[0])

RESULTS FOR k=  7 , with normalization and K-Fold: A
{'total_score': 0.9824152012667722}


In [22]:
### k = 7 | UNNormalized | K-fold: E
k = 7
K_letter = 'E'
input_data = unnormalized_data
## 4 Fold study and selecting fold to test
normalized_testE = fold_dataframe(input_data,K_letter) # function used: fold_dataframe()
## 5 Partitioning
[train_x,train_y,test_x,test_y] = partition_df(normalized_testE) # function used: partition_df()
## 6 Classification 
predictions_for_unnormalized_testE = knn_classify(train_x,train_y,test_x,k)

# 7 Error Analysis
print("RESULTS FOR k= ", k, ", with normalization and K-Fold:", K_letter)
detailed_error_analysis = error_analysis(test_y,predictions_for_unnormalized_testE)
print(detailed_error_analysis[0])

RESULTS FOR k=  7 , with normalization and K-Fold: E
{'total_score': 0.9838319859988333}


In [23]:
#### BONUS PART
## FUNCTION : 

### Weighted kNN classifier
def weighted_knn_classify(train_x,train_y,test_x,k):

    # We should decide how we assign class labels: Majority vote or plurality voting. 
    # There are 16 classes, therefore we will use plurality voting and required vote rate will be 100/16 % + = %6.25+
    # What if there 2 options have rate greater than 6.25.. 

    # We will use Euclidean distance with p=2 as distance metric.
    predicted_list = [] #np.zeros(len(test_x))
    for test_row in test_x: # len(test_x) 
        #test_row = test_x[test_row_index:test_row_index+1][0]
        dist_array = calculate_distances(test_row,train_x)
        # we will find the nearest k neighbours and their target values then we will determine the class for the test row
        # here we can use "argsort" also, however we will try to find them with loop
        """
        for nn in range(k):
            # getting the minimum value of distance list
            min_val = min(dist_array)
            # print(min_val)
            #finding the index of minimum value 
            min_ind = dist_array.index(min_val)
            # we will find the target value of this neighbour
            tvalue = train_y[min_ind][0]
            target_list.append(tvalue)
            # we should update the dist_array value to a huge value so as to not select again
            dist_array[min_ind] = max(dist_array)*99
        """
        nearest_neighbors_indeces = dist_array.argsort()[:k]
        nearest_neighbor_targets = train_y[nearest_neighbors_indeces]
        target_list = [item for sublist in nearest_neighbor_targets for item in sublist]
        ### WEIGHTED KNN ALGO SPECIFIC
        distinct_targets = list(set(target_list))
        weighted_list = []
        for class_i_index in range(len(distinct_targets)):
            item = distinct_targets[class_i_index]
            class_i_indexes = [i for i, x in enumerate(target_list) if x == item]
            weight = 0
            for indices_of_targets in class_i_indexes:
                distance_of_the_row = dist_array[nearest_neighbors_indeces[indices_of_targets]] # dist_array's j'th value | j is nearest_neighbors_indeces's 'indices_of_targets'th value 
                weight = weight + 1/(distance_of_the_row**2)
            weighted_list.append(weight)
        weighted_list = np.array(weighted_list)
        prediction_class = distinct_targets[weighted_list.argsort()[:1][0]]
        # now, we need to find the most frequent element in the target value list
        most_freq_val = prediction_class
        # predicted_list[test_row_index] = most_freq_val
        predicted_list.append(most_freq_val)
    # When we finish predictioning for each test row, we return the prediction results
    return predicted_list

In [24]:
### k = 7 | Normalized | K-fold: E | WEIGHTED KNN
k = 7
K_letter = 'E'
input_data = normalized_data
## 4 Fold study and selecting fold to test
normalized_testE = fold_dataframe(input_data,K_letter) # function used: fold_dataframe()
## 5 Partitioning
[train_x,train_y,test_x,test_y] = partition_df(normalized_testE) # function used: partition_df()
## 6 Classification 
predictions_for_unnormalized_testE = weighted_knn_classify(train_x,train_y,test_x,k)

# 7 Error Analysis
print("RESULTS FOR k= ", k, ", with normalization and K-Fold:", K_letter)
detailed_error_analysis = error_analysis(test_y,predictions_for_unnormalized_testE)
print(detailed_error_analysis[0])

RESULTS FOR k=  7 , with normalization and K-Fold: E
{'total_score': 0.8061505125427119}
